# Week 04 — Effect Size & Business Translation (효과크기/돈으로 번역)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 통계적으로 유의한 차이가 '돈'으로는 얼마인가?
- 작은 uplift라도 규모가 크면 의미가 있는가?

## 2) Why this matters (Theory)
- 통계적 유의성(p<0.05) ≠ 실무 유의성(돈/리스크)
- 효과 크기와 기대 임팩트를 함께 제시해야 의사결정이 쉬움
- 적용 범위(traffic / market / channel)를 명시해야 함

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Uplift → expected monthly revenue impact
✅ uplift를 월 매출 임팩트로 변환하는 연습입니다.

In [ ]:
# Example: compare campaign A vs None as 'new campaign' impact
sub = df[df['campaign'].isin(['A','None'])].copy()
pA = sub[sub['campaign']=='A']['conversions'].mean()
pN = sub[sub['campaign']=='None']['conversions'].mean()
uplift = (pA/pN - 1) if pN>0 else np.nan

# Baseline: average daily revenue under None
baseline_daily = sub[sub['campaign']=='None']['revenue'].fillna(0).mean()
expected_daily_gain = baseline_daily * uplift
expected_monthly_gain = expected_daily_gain * 30

print("conv uplift (A vs None):", uplift)
print("baseline daily revenue (None):", baseline_daily)
print("expected monthly gain (rough):", expected_monthly_gain)

In [ ]:
one_sentence_insight("uplift를 임팩트로 번역했을 때 '실행 가치'가 있는지 한 문장")

### 5.2 Sensitivity: apply only to KR + Ecommerce
✅ 적용 범위를 줄이면 임팩트가 어떻게 변하는지 계산합니다.

In [ ]:
scope = df[(df['country']=='KR') & (df['channel']=='Ecommerce') & (df['campaign'].isin(['A','None']))].copy()
pA = scope[scope['campaign']=='A']['conversions'].mean()
pN = scope[scope['campaign']=='None']['conversions'].mean()
uplift = (pA/pN - 1) if pN>0 else np.nan
baseline_daily = scope[scope['campaign']=='None']['revenue'].fillna(0).mean()
print("uplift:", uplift, "baseline daily:", baseline_daily, "monthly gain:", baseline_daily*uplift*30)

In [ ]:
one_sentence_insight('전체 롤아웃 vs 특정 국가/채널만 적용 중 어떤 전략이 현실적인지 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
